In [2]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

if TMDB_API_KEY and TMDB_API_TOKEN:
    print('✅API KEY and TOKEN are set!')
else:
    print('❌API KEY and TOKEN 404')

✅API KEY and TOKEN are set!


In [7]:
import asyncio
import aiohttp
import pandas as pd
import os
import time
import nest_asyncio

# 주피터 노트북 환경에서 비동기 충돌을 해결합니다.
nest_asyncio.apply()

# TMDB API 토큰을 환경 변수에서 불러옵니다.
# 'TMDB_API_TOKEN'이라는 환경 변수에 여러분의 API 토큰을 설정해야 합니다.
TMDB_API_TOKEN = os.environ.get('TMDB_API_TOKEN')

# API 요청에 필요한 헤더
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {TMDB_API_TOKEN}"
}

# 모든 영화 데이터를 담을 빈 리스트
all_movie_data = []

# 동시 API 요청 수를 제한합니다.
# 이 값을 너무 크게 설정하면 'too many file descriptors' 오류가 발생할 수 있습니다.
# 50으로 줄이면 안정성이 높아집니다.
CONCURRENT_REQUESTS = 50
semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

# 재시도 횟수
MAX_RETRIES = 5

async def fetch_movie_details(session, movie_id):
    """
    비동기적으로 단일 영화의 상세 정보를 가져오는 함수 (재시도 로직 포함)
    """
    # 세마포어를 사용하여 동시 요청 수를 제어합니다.
    async with semaphore:
        url_details = f"https://api.themoviedb.org/3/movie/{movie_id}"
        
        for retry in range(MAX_RETRIES):
            try:
                async with session.get(url_details, headers=headers) as detail_response:
                    if detail_response.status == 200:
                        details = await detail_response.json()
                        movie_data = {
                            'id': details.get('id'),
                            'title': details.get('original_title'), # 'title' 대신 'original_title' 사용
                            'release_date': details.get('release_date'),
                            'budget': details.get('budget'),
                            'revenue': details.get('revenue'),
                            'runtime': details.get('runtime'),
                            'vote_average': details.get('vote_average')
                        }
                        return movie_data
                    elif detail_response.status == 429:
                        print(f"상세 정보 요청 실패: 429 (ID: {movie_id}), API 제한 초과! 60초 대기 후 재시도...")
                        await asyncio.sleep(60) # 429 오류는 1분간 대기
                        continue
                    else:
                        print(f"상세 정보 요청 실패: {detail_response.status} (ID: {movie_id}), 재시도 중... ({retry + 1}/{MAX_RETRIES})")
                        await asyncio.sleep(2 ** retry) # 지수 백오프: 1, 2, 4, 8초 대기
            except aiohttp.ClientError as e:
                print(f"API 요청 중 오류 발생: {e}, 재시도 중... ({retry + 1}/{MAX_RETRIES})")
                await asyncio.sleep(2 ** retry)
        
        # 모든 재시도가 실패했을 경우
        print(f"최종 실패: {movie_id} 영화 정보를 가져올 수 없습니다.")
        return None

async def fetch_movies_by_year(year):
    """
    비동기적으로 특정 연도의 모든 영화 데이터를 가져오는 함수
    """
    print(f"\n--- {year}년 영화 데이터를 수집 중입니다. ---")
    
    # discover/movie 엔드포인트를 위한 파라미터 설정
    params = {
        'language': 'ko-KR',
        'region': 'KR',
        'sort_by': 'popularity.desc',
        'primary_release_year': year,
        'page': 1,
        'vote_count.gte': 10,
        'include_adult': 'false', 
        'with_origin_country': 'KR', # 한국에서 제작한 영화만 포함 (이 필터를 다시 추가)
        'with_release_type': '2|3', # 제한적 개봉과 일반 개봉을 모두 포함
    }
    
    tasks = []
    
    async with aiohttp.ClientSession(headers=headers) as session:
        # 첫 페이지에서 전체 페이지 수 가져오기
        response = await session.get("https://api.themoviedb.org/3/discover/movie", params=params)
        data = await response.json()
        total_pages = data.get('total_pages', 1)

        # 각 페이지별로 영화 ID 가져오기
        for page in range(1, total_pages + 1):
            params['page'] = page
            response = await session.get("https://api.themoviedb.org/3/discover/movie", params=params)
            data = await response.json()
            movie_ids = [movie['id'] for movie in data.get('results', [])]
            
            # 각 영화 ID에 대한 상세 정보 요청 작업을 생성
            for movie_id in movie_ids:
                task = asyncio.create_task(fetch_movie_details(session, movie_id))
                tasks.append(task)
            print(f"  > {year}년 {page}/{total_pages} 페이지 영화 {len(movie_ids)}개 작업 등록 완료")
            
        # 모든 비동기 작업이 완료될 때까지 기다림
        results = await asyncio.gather(*tasks)
        
        # 유효한 데이터만 필터링하여 리스트에 추가
        for movie_data in results:
            if movie_data:
                all_movie_data.append(movie_data)

async def main():
    """
    메인 함수: 2005년부터 2024년까지 비동기 수집을 시작합니다.
    """
    start_time = time.time()
    await asyncio.gather(*(fetch_movies_by_year(year) for year in range(2005, 2025)))
    
    # 모든 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame(all_movie_data)
    
    # 'release_date' 열을 날짜/시간 형식으로 변환하여 올바르게 정렬
    df['release_date'] = pd.to_datetime(df['release_date'])
    # 'release_date'를 기준으로 오름차순 정렬
    df = df.sort_values(by='release_date')
    
    # DataFrame을 CSV 파일로 저장 (한글 깨짐 방지)
    df.to_csv("korean_movies_2005_2024_theatrical_full.csv", index=False, encoding='utf-8-sig')
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print("\n--- 모든 데이터 수집 및 저장 완료 ---")
    print(f"총 {len(df)}개 영화의 데이터가 'korean_movies_2005_2024_theatrical_full.csv' 파일에 저장되었습니다.")
    print(f"총 소요 시간: {elapsed_time:.2f}초")

if __name__ == "__main__":
    await main()



--- 2005년 영화 데이터를 수집 중입니다. ---

--- 2006년 영화 데이터를 수집 중입니다. ---

--- 2007년 영화 데이터를 수집 중입니다. ---

--- 2008년 영화 데이터를 수집 중입니다. ---

--- 2009년 영화 데이터를 수집 중입니다. ---

--- 2010년 영화 데이터를 수집 중입니다. ---

--- 2011년 영화 데이터를 수집 중입니다. ---

--- 2012년 영화 데이터를 수집 중입니다. ---

--- 2013년 영화 데이터를 수집 중입니다. ---

--- 2014년 영화 데이터를 수집 중입니다. ---

--- 2015년 영화 데이터를 수집 중입니다. ---

--- 2016년 영화 데이터를 수집 중입니다. ---

--- 2017년 영화 데이터를 수집 중입니다. ---

--- 2018년 영화 데이터를 수집 중입니다. ---

--- 2019년 영화 데이터를 수집 중입니다. ---

--- 2020년 영화 데이터를 수집 중입니다. ---

--- 2021년 영화 데이터를 수집 중입니다. ---

--- 2022년 영화 데이터를 수집 중입니다. ---

--- 2023년 영화 데이터를 수집 중입니다. ---

--- 2024년 영화 데이터를 수집 중입니다. ---
  > 2010년 1/3 페이지 영화 20개 작업 등록 완료
  > 2022년 1/3 페이지 영화 20개 작업 등록 완료
  > 2013년 1/4 페이지 영화 20개 작업 등록 완료
  > 2008년 1/3 페이지 영화 20개 작업 등록 완료
  > 2009년 1/3 페이지 영화 20개 작업 등록 완료
  > 2017년 1/5 페이지 영화 20개 작업 등록 완료
  > 2015년 1/4 페이지 영화 20개 작업 등록 완료
  > 2019년 1/5 페이지 영화 20개 작업 등록 완료
  > 2005년 1/3 페이지 영화 20개 작업 등록 완료
  > 2020년 1/4 페이지 영화 20개 작업 등록 완료
  > 2012년 1/3 페이지 영화

In [5]:
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOFIC(KOBIS)에서 '한국 제작 + 국내 극장 개봉' 영화만 후보 단계부터 선별하여
TMDB 스타일 스키마 CSV 생성 (revenue=국내 최종 누적매출, audience_total=최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ===== 사용자 설정 =====
START_DATE = "2005-01-01"
END_DATE   = "2025-12-31"

OUT_DIR        = "./data_processed"
CANDIDATES_CSV = f"{OUT_DIR}/kofic_candidates_kr_2005_2025.csv"   # 한국만 담긴 후보
OUT_CSV        = f"{OUT_DIR}/kofic_domestic_kr_2005_2025.csv"     # 최종 결과

TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.15  # 너무 낮추면 429 위험

# 주간 박스오피스 기반 최종 누적 추정 파라미터
JUMP_STEPS_DAYS     = [0, 14, 28, 56, 84]  # 개봉 후 0/2/4/8/12주
LOCAL_REFINE_RADIUS = 7                    # 마지막 앵커 ±1주 보정
MIN_ABS_DELTA  = 50_000_000               # 매출 증가 최소치(원)
MIN_REL_DELTA  = 0.01                     # 증가율 임계(1%)
PLATEAU_STREAK = 2                        # 2회 연속 정체 → 종료

# ===== 준비 =====
load_dotenv()
KOFIC_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
if not KOFIC_KEY:
    raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")
os.makedirs(OUT_DIR, exist_ok=True)

BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"

def kobis_get(path, params, retry=RETRY):
    q = {"key": KOFIC_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last_err = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            data = r.json()
            if isinstance(data, dict) and data.get("faultInfo"):
                fi  = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last_err = e
            if i == retry - 1:
                print("❌ Request failed:", e, "\nURL:", url)
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError(f"unreachable: {last_err}")

# ===== 공통 유틸 =====
def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    if len(s) != 8 or not s.isdigit(): return pd.NA
    return f"{s[:4]}-{s[4:6]}-{s[6:]}"

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list):
        return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

def sunday_of_week(d: date) -> date:
    # 월=0..일=6 → 일요일로 보정
    return d + timedelta(days=(6 - d.weekday()))

def is_korean_production(info):
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def in_period_open(info, start=START_DATE, end=END_DATE):
    openDt = info.get("openDt")
    if not openDt: return False
    iso = to_iso_from_yyyymmdd(openDt)
    if pd.isna(iso): return False
    return (iso >= start) and (iso <= end)

# ===== (옵션2) 공통코드에서 ‘한국’ 8자리 코드 조회 =====
# 에러 메시지에 ‘공통코드220310’이 언급되는 경우가 있어 다양한 후보 comCode를 순회 탐색
CODE_TABLE_CANDIDATES = ["220310", "2204", "22003", "2203", "220301", "220201"]

def get_korea_code(verbose=True):
    for cc in CODE_TABLE_CANDIDATES:
        try:
            js = kobis_get("/code/searchCodeList", {"comCode": cc})
        except Exception as e:
            if verbose:
                print(f"  · 공통코드 {cc} 조회 실패: {e}")
            continue

        # 응답 구조 가변 대응
        code_list = (
            (js.get("codeListResult") or {}).get("codeList")
            or js.get("codes")
            or js.get("codeList")
            or []
        )

        found = None
        for c in code_list:
            name_ko = c.get("korNm") or c.get("codeNm") or c.get("korName") or ""
            name_en = (c.get("engNm") or c.get("engName") or "").upper()
            code    = c.get("fullCd") or c.get("code") or c.get("cd") or ""
            if (("한국" in name_ko) or ("대한민국" in name_ko)
                or ("KOREA" in name_en) or ("REPUBLIC OF KOREA" in name_en)):
                if isinstance(code, str) and len(code) == 8 and code.isdigit():
                    found = code; break

        if found:
            if verbose:
                print(f"✅ 한국 코드 발견 (comCode={cc}): {found}")
            return found

    if verbose:
        print("⚠️ 공통코드 테이블에서 한국 코드(8자리)를 찾지 못했습니다. 응답 필터 폴백을 사용합니다.")
    return None

# ===== 1) 후보(연도별) 수집: ‘서버 필터(한국)’ 우선, 실패시 폴백(응답에서 한국만 선별) =====
KR_LABELS = {"한국","대한민국","Korea","South Korea","Republic of Korea"}

def find_candidates_by_year(year: int, kr_code: str | None):
    y = str(year)
    # 우선 ‘연/기간’ 3가지 패턴으로 프로브 → 성공 조합으로 페이징
    base_patterns = [
        {"openStartDt": y,          "openEndDt": y          },
        {"openStartDt": y                                  },
        {"openStartDt": f"{y}0101", "openEndDt": f"{y}1231"},
    ]

    def probe(params):
        js = kobis_get("/movie/searchMovieList", {**params, "itemPerPage": 10, "curPage": 1})
        res = js.get("movieListResult", {}) or {}
        tot = res.get("totCnt")
        return int(tot) if (isinstance(tot, (int,str)) and str(tot).isdigit()) else 0

    chosen = None
    mode = "ANY"

    # 1) repNationCd=한국코드 사용 프로브
    if kr_code:
        for p in base_patterns:
            params = {**p, "repNationCd": kr_code}
            if probe(params) > 0:
                chosen = params
                mode = "KCODE"
                break

    # 2) 실패 시 무필터로 프로브
    if chosen is None:
        for p in base_patterns:
            if probe(p) > 0:
                chosen = p
                mode = "ANY"
                break

    if chosen is None:
        print(f"  · {year}: 후보 0건 (건너뜀)")
        return []

    out, per_page, cur_page, total_seen = [], 100, 1, None
    while True:
        params = {**chosen, "itemPerPage": per_page, "curPage": cur_page}
        js = kobis_get("/movie/searchMovieList", params)
        res = js.get("movieListResult", {}) or {}
        lst = res.get("movieList", []) or []
        tot = res.get("totCnt", None)
        if total_seen is None and tot is not None:
            total_seen = int(tot)

        if not lst: break

        for x in lst:
            mc  = x.get("movieCd")
            rep = (x.get("repNationNm") or "").strip()
            if not mc:
                continue
            if (mode == "KCODE") or (rep in KR_LABELS):
                out.append((mc, x.get("movieNm"), x.get("openDt")))

        # repNationCd를 쓴 경우에는 totCnt에 신뢰
        if (mode == "KCODE") and (total_seen is not None) and (len(out) >= total_seen):
            break

        cur_page += 1
        time.sleep(SLEEP)

    print(f"  · {year}: mode={mode:<6}, collected(K-only)={len(out):>5}")
    return out

def build_candidates(save_path=CANDIDATES_CSV, start=START_DATE, end=END_DATE):
    s_year, e_year = int(start[:4]), int(end[:4])
    print("연도별 후보 수집(서버 한국코드 우선)…")
    kr_code = get_korea_code(verbose=True)
    cands = []
    for y in range(s_year, e_year+1):
        cands.extend(find_candidates_by_year(y, kr_code))
    df = pd.DataFrame(cands, columns=["movieCd","movieNm","openDt"]).drop_duplicates("movieCd")
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"📄 후보 저장(한국만): {save_path} (고유 {len(df):,}편)")
    return df

# ===== 2) 주간 박스오피스 기반 누적(매출/관객) 추정 =====
weekly_cache = {}

def weekly_items(target_yyyymmdd: str):
    if target_yyyymmdd in weekly_cache:
        return weekly_cache[target_yyyymmdd]
    js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                   {"targetDt": target_yyyymmdd, "weekGb": "0"})
    items = js.get("boxOfficeResult", {}).get("weeklyBoxOfficeList", []) or []
    weekly_cache[target_yyyymmdd] = items
    return items

def get_acc_from_weekly_by_code(movieCd: str, target_date: date):
    tgt = sunday_of_week(target_date).strftime("%Y%m%d")
    for it in weekly_items(tgt):
        if it.get("movieCd") == movieCd:
            s = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            a = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            return s, a
    return 0, 0

def intelligent_final_acc(movieCd: str, open_yyyymmdd: str):
    if not open_yyyymmdd or len(str(open_yyyymmdd)) != 8:
        return (pd.NA, pd.NA)
    base = datetime.strptime(open_yyyymmdd, "%Y%m%d").date()

    best_sales, best_audi = 0.0, 0.0
    plateau_cnt, prev = 0, 0.0

    for dd in JUMP_STEPS_DAYS:
        anchor = base + timedelta(days=dd)
        s, a = get_acc_from_weekly_by_code(movieCd, anchor)
        if s > best_sales: best_sales = s
        if a > best_audi:  best_audi  = a

        delta = best_sales - prev
        rel = (delta/prev) if prev > 0 else 1.0
        plateau_cnt = plateau_cnt + 1 if (delta < MIN_ABS_DELTA and rel < MIN_REL_DELTA) else 0
        prev = best_sales

        if plateau_cnt >= PLATEAU_STREAK:
            for d in range(-LOCAL_REFINE_RADIUS, LOCAL_REFINE_RADIUS+1, 7):
                anchor2 = anchor + timedelta(days=d)
                s2, a2 = get_acc_from_weekly_by_code(movieCd, anchor2)
                if s2 > best_sales: best_sales = s2
                if a2 > best_audi:  best_audi  = a2
            break

    return (pd.NA if best_sales == 0 else best_sales,
            pd.NA if best_audi  == 0 else best_audi)

# ===== 3) 상세 → TMDB 스타일 행 빌드 =====
def fetch_movie_info(movieCd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": movieCd})
    return js.get("movieInfoResult", {}).get("movieInfo", {}) or {}

def to_tmdb_row(info, salesAcc_final, audiAcc_final):
    movieCd   = info.get("movieCd")
    movieNm   = info.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt")  # yyyymmdd
    release_date = to_iso_from_yyyymmdd(openDt)

    dt = pd.to_datetime(release_date, errors="coerce") if pd.notna(release_date) else pd.NaT
    release_year  = (dt.year  if pd.notna(dt) else pd.NA)
    release_month = (dt.month if pd.notna(dt) else pd.NA)

    showTm = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": movieCd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": release_date,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": pd.to_numeric(salesAcc_final),  # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": release_year,
        "release_month": release_month,
        "audience_total": pd.to_numeric(audiAcc_final),  # 국내 최종 누적관객
    }

def enrich_and_save(candidates_csv=CANDIDATES_CSV, out_csv=OUT_CSV):
    # 후보 로드 or 생성
    if os.path.exists(candidates_csv):
        cand = pd.read_csv(candidates_csv, dtype=str)
        print(f"📄 후보 로드: {candidates_csv} (rows={len(cand):,})")
    else:
        cand = build_candidates()

    rows = []
    with tqdm(total=len(cand), desc="Detail + domestic totals") as bar:
        for _, r in cand.iterrows():
            movieCd = str(r["movieCd"])
            try:
                info = fetch_movie_info(movieCd)
            except Exception:
                bar.update(1); continue

            # 상세에서도 한국 제작 & 기간 내 개봉 확인 (이중 안전망)
            if (not is_korean_production(info)) or (not in_period_open(info)):
                bar.update(1); continue

            try:
                s_final, a_final = intelligent_final_acc(movieCd, info.get("openDt"))
            except Exception:
                s_final, a_final = (pd.NA, pd.NA)

            rows.append(to_tmdb_row(info, s_final, a_final))
            bar.update(1)

    df = pd.DataFrame(rows)
    if not df.empty:
        df["runtime"]        = pd.to_numeric(df["runtime"], errors="coerce")
        df["revenue"]        = pd.to_numeric(df["revenue"], errors="coerce")
        df["audience_total"] = pd.to_numeric(df["audience_total"], errors="coerce")
        df["release_year"]   = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")
        df["release_month"]  = pd.to_numeric(df["release_month"], errors="coerce").astype("Int64")

    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]
    df = df.reindex(columns=cols)

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {out_csv} (rows={len(df):,})")
    print(f"🗃 weekly_cache 크기(앵커 호출 수): {len(weekly_cache):,}")

# ===== 실행 =====
if __name__ == "__main__":
    if not os.path.exists(CANDIDATES_CSV):
        build_candidates()
    enrich_and_save()


연도별 후보 수집(서버 한국코드 우선)…
✅ 한국 코드 발견 (comCode=220310): 22041011
  · 2005: mode=KCODE , collected(K-only)=   82
  · 2006: mode=KCODE , collected(K-only)=  117
  · 2007: mode=KCODE , collected(K-only)=  139
  · 2008: mode=KCODE , collected(K-only)=  157
  · 2009: mode=KCODE , collected(K-only)=  123
  · 2010: mode=KCODE , collected(K-only)=  155
  · 2011: mode=KCODE , collected(K-only)=  168
  · 2012: mode=KCODE , collected(K-only)=  177
  · 2013: mode=KCODE , collected(K-only)=  186
  · 2014: mode=KCODE , collected(K-only)=  218
  · 2015: mode=KCODE , collected(K-only)=  257
  · 2016: mode=KCODE , collected(K-only)=  338
  · 2017: mode=KCODE , collected(K-only)=  495
  · 2018: mode=KCODE , collected(K-only)=  661
  · 2019: mode=KCODE , collected(K-only)=  697
  · 2020: mode=KCODE , collected(K-only)=  782
  · 2021: mode=KCODE , collected(K-only)=  818
  · 2022: mode=KCODE , collected(K-only)=  772
  · 2023: mode=KCODE , collected(K-only)=  665
  · 2024: mode=KCODE , collected(K-only)=  618

Detail + domestic totals:  27%|██▋       | 2138/7974 [32:53<3:28:27,  2.14s/it] 

❌ Request failed: KOBIS fault: 키의 하루 이용량을 초과하였습니다. (code=320011)
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=a3c5aea7d99b9dca621b49723c85b8e3&targetDt=20170604&weekGb=0 
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=a3c5aea7d99b9dca621b49723c85b8e3&targetDt=20170604&weekGb=0


Detail + domestic totals:  27%|██▋       | 2139/7974 [32:56<4:09:03,  2.56s/it]

❌ Request failed: KOBIS fault: 키의 하루 이용량을 초과하였습니다. (code=320011)
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20173251 
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20173251


...

❌ Request failed: KOBIS fault: 키의 하루 이용량을 초과하였습니다. (code=320011)
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20177925 
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20177925


Detail + domestic totals:  27%|██▋       | 2176/7974 [35:30<7:36:33,  4.72s/it]

❌ Request failed: KOBIS fault: 키의 하루 이용량을 초과하였습니다. (code=320011)
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20138561 
URL: https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a3c5aea7d99b9dca621b49723c85b8e3&movieCd=20138561


Detail + domestic totals:  27%|██▋       | 2176/7974 [35:34<1:34:46,  1.02it/s]


KeyboardInterrupt: 

In [ ]:
import requests
import json

# 여기에 발급받은 KOFIC API 키를 입력하세요.
API_KEY = ""

# 테스트를 위한 API 엔드포인트
url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={API_KEY}&targetDt=20230101"

try:
    response = requests.get(url)
    
    # HTTP 상태 코드 확인 (200 OK)
    if response.status_code == 200:
        data = json.loads(response.text)
        
        # API 응답에 'faultInfo'가 있는지 확인
        if 'faultInfo' in data:
            fault_info = data['faultInfo']
            error_code = fault_info.get('errorCode')
            message = fault_info.get('message')
            
            # 오류 메시지를 통해 키 유효성 판단
            if error_code == '3209' or '유효하지 않은 키' in message:
                print("❌ API 키가 유효하지 않습니다.")
                print(f"오류 코드: {error_code}, 메시지: {message}")
            else:
                print("✅ API 호출은 성공했으나, 알 수 없는 오류가 발생했습니다.")
                print(f"오류 코드: {error_code}, 메시지: {message}")
        else:
            print("🎉 API 키가 유효합니다! API 호출에 성공했습니다.")
            # 성공 응답 데이터 일부 출력
            boxoffice_list = data.get('boxOfficeResult', {}).get('dailyBoxOfficeList', [])
            if boxoffice_list:
                print(f"첫 번째 영화: {boxoffice_list[0]['movieNm']}")
            
    else:
        print(f"❌ API 호출에 실패했습니다. HTTP 상태 코드: {response.status_code}")
        print("API 키를 다시 확인해주세요.")

except requests.exceptions.RequestException as e:
    print(f"❌ 요청 중 오류 발생: {e}")
    print("네트워크 연결 또는 URL을 확인해주세요.")

🎉 API 키가 유효합니다! API 호출에 성공했습니다.
첫 번째 영화: 아바타: 물의 길


In [5]:
# kobis_weekly_to_tmdb_csv.py
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOBIS(=KOFIC OpenAPI)에서 '주간 박스오피스 Top10에 한 번이라도 등장'한 영화만 선별
→ 한국 제작만 필터 → TMDB 스타일 스키마 CSV 생성
(revenue = 국내 최종 누적매출, audience_total = 국내 최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ========= 기간/경로 설정 =========
START = date(2005, 1, 1)
END   = date(2025,12,31)

OUT_DIR       = "./data_processed"
WEEKLY_PICKLE = f"{OUT_DIR}/kobis_weekly_pool.pkl"      # 주간 스캔 결과 캐시
OUT_PARTIAL   = f"{OUT_DIR}/kobis_weekly_partial.csv"   # 메타 누적(재시작용)
OUT_FINAL     = f"{OUT_DIR}/kobis_weekly_final.csv"     # 최종 TMDB형

# 하루에 처리할 movieInfo 개수(쿼터 따라 조정)
MOVIES_PER_RUN = 200

# 요청/재시도
TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.12

# ========= 인증 =========
load_dotenv()
KOBIS_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"
os.makedirs(OUT_DIR, exist_ok=True)

# ========= 공통 HTTP =========
def kobis_get(path, params, retry=RETRY):
    q = {"key": KOBIS_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429,500,502,503,504):
                time.sleep(min(2**i,8)); continue
            r.raise_for_status()
            data = r.json()
            if data.get("faultInfo"):
                fi = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last = e
            if i == retry-1: raise
            time.sleep(min(2**i,8))
    raise RuntimeError(last)

# ========= 유틸 =========
def sundays(start: date, end: date):
    # start 기준 그 주 일요일(weekGb=0 사용)
    s = start + timedelta(days=(6 - start.weekday()))
    while s <= end:
        yield s
        s += timedelta(days=7)

def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    return f"{s[:4]}-{s[4:6]}-{s[6:]}" if len(s) >= 8 else pd.NA

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list): return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

# ========= 1) 주간 Top10 스캔: 등장 영화 + 최종 누적 =========
def weekly_scan():
    """
    주간 박스오피스(Top10) 전체 주차 스캔 → movieCd별 최댓값 누적매출/관객 및 기본 정보 모음
    반환: dict[movieCd] = {max_sales, max_audi, movieNm, openDt, weeks}
    """
    pool = {}  # movieCd -> dict
    for s in tqdm(list(sundays(START, END)), desc="Weekly scan (Top10)"):
        js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                       {"targetDt": s.strftime("%Y%m%d"), "weekGb": "0"})
        items = js.get("boxOfficeResult",{}).get("weeklyBoxOfficeList",[]) or []
        for it in items:
            cd  = it.get("movieCd")
            if not cd: continue
            nm  = it.get("movieNm")
            odt = it.get("openDt")  # yyyymmdd
            salesAcc = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            audiAcc  = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            rec = pool.get(cd, {"max_sales":0, "max_audi":0, "movieNm":nm, "openDt":odt, "weeks":0})
            rec["max_sales"] = max(rec["max_sales"], salesAcc)
            rec["max_audi"]  = max(rec["max_audi"],  audiAcc)
            rec["weeks"]     = rec["weeks"] + 1
            if not rec.get("movieNm"):  rec["movieNm"] = nm
            if not rec.get("openDt"):   rec["openDt"]  = odt
            pool[cd] = rec
    pd.to_pickle(pool, WEEKLY_PICKLE)
    return pool

# ========= 2) movieInfo 메타(한국 제작 필터) + TMDB 스키마로 변환 =========
def fetch_movie_info(cd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": cd})
    return js.get("movieInfoResult",{}).get("movieInfo",{}) or {}

def is_korean_production(info) -> bool:
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def to_tmdb_row(cd, pool_rec, info):
    movieNm   = info.get("movieNm") or pool_rec.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt") or pool_rec.get("openDt")
    iso       = to_iso_from_yyyymmdd(openDt)
    dt        = pd.to_datetime(iso, errors="coerce") if pd.notna(iso) else pd.NaT

    showTm    = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": cd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": iso,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": float(pool_rec["max_sales"]),       # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": (dt.year  if pd.notna(dt) else pd.NA),
        "release_month": (dt.month if pd.notna(dt) else pd.NA),
        "audience_total": float(pool_rec["max_audi"]), # 국내 최종 누적관객
    }

# ========= 3) 실행(재시작 가능) =========
def main():
    if not KOBIS_KEY:
        raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")

    # 1) 주간 스캔(캐시 활용)
    if os.path.exists(WEEKLY_PICKLE):
        pool = pd.read_pickle(WEEKLY_PICKLE)
    else:
        pool = weekly_scan()

    # 주간에 한 번이라도 잡힌 영화만(=누적매출 추적 가능)
    candidates = [cd for cd, rec in pool.items() if rec.get("max_sales",0) > 0]

    # 2) 이미 수집한 partial 읽기(재시작)
    done = set()
    if os.path.exists(OUT_PARTIAL):
        prev = pd.read_csv(OUT_PARTIAL, dtype=str)
        if "movie_id" in prev.columns:
            done = set(prev["movie_id"].astype(str).tolist())

    todo = [cd for cd in candidates if cd not in done]
    if not todo:
        print("✅ 모두 처리됨. 최종 파일만 정리합니다.")
    else:
        batch = todo[:MOVIES_PER_RUN]
        print(f"이번 배치: {len(batch)}편 / 남은 {len(todo)-len(batch)}편")

        rows = []
        for cd in tqdm(batch, desc="movieInfo + merge (KR only)"):
            try:
                info = fetch_movie_info(cd)
                # 한국 제작 필터 + 개봉일(기간 내 안전망)
                if not is_korean_production(info):
                    continue
                odt = info.get("openDt") or pool[cd].get("openDt")
                if not odt or len(str(odt)) != 8:
                    continue
                # 기간 안전망 (열린 날짜 기준)
                y, m, d = int(odt[:4]), int(odt[4:6]), int(odt[6:8])
                opened = date(y,m,d)
                if opened < START or opened > END:
                    continue

                row = to_tmdb_row(cd, pool[cd], info)
                rows.append(row)
            except Exception as e:
                print("skip", cd, e)

        if rows:
            df = pd.DataFrame(rows)
            header = not os.path.exists(OUT_PARTIAL)
            df.to_csv(OUT_PARTIAL, index=False, mode=("w" if header else "a"),
                      header=header, encoding="utf-8-sig")

    # 3) 최종 정리/저장(TMDB 스키마)
    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]

    part = pd.read_csv(OUT_PARTIAL, dtype=str) if os.path.exists(OUT_PARTIAL) else pd.DataFrame(columns=cols)
    if not part.empty:
        part["release_year"]   = pd.to_numeric(part["release_year"], errors="coerce").astype("Int64")
        part["release_month"]  = pd.to_numeric(part["release_month"], errors="coerce").astype("Int64")
        part["runtime"]        = pd.to_numeric(part["runtime"], errors="coerce")
        part["revenue"]        = pd.to_numeric(part["revenue"], errors="coerce")
        part["audience_total"] = pd.to_numeric(part["audience_total"], errors="coerce")
        part = part.drop_duplicates(subset=["movie_id"]).sort_values(["release_year","release_month","title"])

    part.reindex(columns=cols).to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_FINAL} (rows={len(part):,})")

if __name__ == "__main__":
    main()


이번 배치: 200편 / 남은 2849편


movieInfo + merge (KR only): 100%|██████████| 200/200 [01:39<00:00,  2.00it/s]

✅ 저장: ./data_processed/kobis_weekly_final.csv (rows=449)


In [1]:
# kobis_weekly_to_tmdb_csv.py
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOBIS(=KOFIC OpenAPI)에서 '주간 박스오피스 Top10에 한 번이라도 등장'한 영화만 선별
→ 한국 제작만 필터 → TMDB 스타일 스키마 CSV 생성
(revenue = 국내 최종 누적매출, audience_total = 국내 최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ========= 기간/경로 설정 =========
START = date(2024, 1, 1)
END   = date(2025,12,31)

OUT_DIR       = "./data_processed"
WEEKLY_PICKLE = f"{OUT_DIR}/kobis_weekly_pool.pkl"      # 주간 스캔 결과 캐시
OUT_PARTIAL   = f"{OUT_DIR}/kobis_weekly_partial.csv"   # 메타 누적(재시작용)
OUT_FINAL     = f"{OUT_DIR}/kobis_weekly_final_2024.csv"     # 최종 TMDB형

# 하루에 처리할 movieInfo 개수(쿼터 따라 조정)
MOVIES_PER_RUN = 800

# 요청/재시도
TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.12

# ========= 인증 =========
load_dotenv()
KOBIS_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"
os.makedirs(OUT_DIR, exist_ok=True)

# ========= 공통 HTTP =========
def kobis_get(path, params, retry=RETRY):
    q = {"key": KOBIS_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429,500,502,503,504):
                time.sleep(min(2**i,8)); continue
            r.raise_for_status()
            data = r.json()
            if data.get("faultInfo"):
                fi = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last = e
            if i == retry-1: raise
            time.sleep(min(2**i,8))
    raise RuntimeError(last)

# ========= 유틸 =========
def sundays(start: date, end: date):
    # start 기준 그 주 일요일(weekGb=0 사용)
    s = start + timedelta(days=(6 - start.weekday()))
    while s <= end:
        yield s
        s += timedelta(days=7)

def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    return f"{s[:4]}-{s[4:6]}-{s[6:]}" if len(s) >= 8 else pd.NA

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list): return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

# ========= 1) 주간 Top10 스캔: 등장 영화 + 최종 누적 =========
def weekly_scan():
    """
    주간 박스오피스(Top10) 전체 주차 스캔 → movieCd별 최댓값 누적매출/관객 및 기본 정보 모음
    반환: dict[movieCd] = {max_sales, max_audi, movieNm, openDt, weeks}
    """
    pool = {}  # movieCd -> dict
    for s in tqdm(list(sundays(START, END)), desc="Weekly scan (Top10)"):
        js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                       {"targetDt": s.strftime("%Y%m%d"), "weekGb": "0"})
        items = js.get("boxOfficeResult",{}).get("weeklyBoxOfficeList",[]) or []
        for it in items:
            cd  = it.get("movieCd")
            if not cd: continue
            nm  = it.get("movieNm")
            odt = it.get("openDt")  # yyyymmdd
            salesAcc = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            audiAcc  = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            rec = pool.get(cd, {"max_sales":0, "max_audi":0, "movieNm":nm, "openDt":odt, "weeks":0})
            rec["max_sales"] = max(rec["max_sales"], salesAcc)
            rec["max_audi"]  = max(rec["max_audi"],  audiAcc)
            rec["weeks"]     = rec["weeks"] + 1
            if not rec.get("movieNm"):  rec["movieNm"] = nm
            if not rec.get("openDt"):   rec["openDt"]  = odt
            pool[cd] = rec
    pd.to_pickle(pool, WEEKLY_PICKLE)
    return pool

# ========= 2) movieInfo 메타(한국 제작 필터) + TMDB 스키마로 변환 =========
def fetch_movie_info(cd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": cd})
    return js.get("movieInfoResult",{}).get("movieInfo",{}) or {}

def is_korean_production(info) -> bool:
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def to_tmdb_row(cd, pool_rec, info):
    movieNm   = info.get("movieNm") or pool_rec.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt") or pool_rec.get("openDt")
    iso       = to_iso_from_yyyymmdd(openDt)
    dt        = pd.to_datetime(iso, errors="coerce") if pd.notna(iso) else pd.NaT

    showTm    = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": cd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": iso,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": float(pool_rec["max_sales"]),       # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": (dt.year  if pd.notna(dt) else pd.NA),
        "release_month": (dt.month if pd.notna(dt) else pd.NA),
        "audience_total": float(pool_rec["max_audi"]), # 국내 최종 누적관객
    }

# ========= 3) 실행(재시작 가능) =========
def main():
    if not KOBIS_KEY:
        raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")

    # 1) 주간 스캔(캐시 활용)
    if os.path.exists(WEEKLY_PICKLE):
        pool = pd.read_pickle(WEEKLY_PICKLE)
    else:
        pool = weekly_scan()

    # 주간에 한 번이라도 잡힌 영화만(=누적매출 추적 가능)
    candidates = [cd for cd, rec in pool.items() if rec.get("max_sales",0) > 0]

    # 2) 이미 수집한 partial 읽기(재시작)
    done = set()
    if os.path.exists(OUT_PARTIAL):
        prev = pd.read_csv(OUT_PARTIAL, dtype=str)
        if "movie_id" in prev.columns:
            done = set(prev["movie_id"].astype(str).tolist())

    todo = [cd for cd in candidates if cd not in done]
    if not todo:
        print("✅ 모두 처리됨. 최종 파일만 정리합니다.")
    else:
        batch = todo[:MOVIES_PER_RUN]
        print(f"이번 배치: {len(batch)}편 / 남은 {len(todo)-len(batch)}편")

        rows = []
        for cd in tqdm(batch, desc="movieInfo + merge (KR only)"):
            try:
                info = fetch_movie_info(cd)
                # 한국 제작 필터 + 개봉일(기간 내 안전망)
                if not is_korean_production(info):
                    continue
                odt = info.get("openDt") or pool[cd].get("openDt")
                if not odt or len(str(odt)) != 8:
                    continue
                # 기간 안전망 (열린 날짜 기준)
                y, m, d = int(odt[:4]), int(odt[4:6]), int(odt[6:8])
                opened = date(y,m,d)
                if opened < START or opened > END:
                    continue

                row = to_tmdb_row(cd, pool[cd], info)
                rows.append(row)
            except Exception as e:
                print("skip", cd, e)

        if rows:
            df = pd.DataFrame(rows)
            header = not os.path.exists(OUT_PARTIAL)
            df.to_csv(OUT_PARTIAL, index=False, mode=("w" if header else "a"),
                      header=header, encoding="utf-8-sig")

    # 3) 최종 정리/저장(TMDB 스키마)
    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]

    part = pd.read_csv(OUT_PARTIAL, dtype=str) if os.path.exists(OUT_PARTIAL) else pd.DataFrame(columns=cols)
    if not part.empty:
        part["release_year"]   = pd.to_numeric(part["release_year"], errors="coerce").astype("Int64")
        part["release_month"]  = pd.to_numeric(part["release_month"], errors="coerce").astype("Int64")
        part["runtime"]        = pd.to_numeric(part["runtime"], errors="coerce")
        part["revenue"]        = pd.to_numeric(part["revenue"], errors="coerce")
        part["audience_total"] = pd.to_numeric(part["audience_total"], errors="coerce")
        part = part.drop_duplicates(subset=["movie_id"]).sort_values(["release_year","release_month","title"])

    part.reindex(columns=cols).to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_FINAL} (rows={len(part):,})")

if __name__ == "__main__":
    main()


Weekly scan (Top10): 100%|██████████| 104/104 [01:48<00:00,  1.05s/it]


이번 배치: 274편 / 남은 0편


movieInfo + merge (KR only): 100%|██████████| 274/274 [02:52<00:00,  1.59it/s]

✅ 저장: ./data_processed/kobis_weekly_final_2024.csv (rows=97)


In [5]:
import pandas as pd

# 파일 경로
file_path = 'data_processed/kobis_weekly_clean_exploded.csv'

try:
    # CSV 파일 로드
    df = pd.read_csv(file_path)

    # DataFrame 정보 출력으로 컬럼명 확인
    print("--- DataFrame 정보 ---")
    df.info()

    # 1. 결측치(NaN) 확인
    print("\n--- 결측치(NaN) 확인 ---")
    print(df.isna().sum())

    # 2. 특정 컬럼의 0 값 확인
    print("\n--- 특정 컬럼 0 값 확인 ---")
    revenue_zeros = (df['revenue'] == 0).sum()
    audience_total_zeros = (df['audience_total'] == 0).sum()
    # 'genres' 컬럼은 문자열일 수 있으므로 '0'도 함께 확인
    genres_zeros = (df['genres'] == 0).sum() + (df['genres'] == '0').sum()

    print(f"revenue 컬럼의 0 값 개수: {revenue_zeros}")
    print(f"audience_total 컬럼의 0 값 개수: {audience_total_zeros}")
    print(f"genres 컬럼의 0 값 개수: {genres_zeros}")

except FileNotFoundError:
    print(f"\n오류: '{file_path}' 파일을 찾을 수 없습니다. 파일명이나 경로를 다시 확인해주세요.")
except KeyError as e:
    print(f"\n오류: {e} 컬럼이 DataFrame에 없습니다. 컬럼명을 다시 확인해주세요.")

--- DataFrame 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2027 entries, 0 to 2026
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              2027 non-null   int64  
 1   title                 2027 non-null   object 
 2   original_title        2027 non-null   object 
 3   original_language     2027 non-null   object 
 4   release_date          2027 non-null   object 
 5   runtime               2023 non-null   float64
 6   budget                0 non-null      float64
 7   revenue               2027 non-null   float64
 8   vote_average          0 non-null      float64
 9   vote_count            0 non-null      float64
 10  popularity            0 non-null      float64
 11  genres                2027 non-null   object 
 12  production_companies  2027 non-null   object 
 13  production_countries  2027 non-null   object 
 14  release_year          2027 non-null   int64  
 15  

In [10]:
import pandas as pd

# 파일 경로
file_path = 'data_processed/kobis_with_credits_ko.csv'
output_file_path = 'data_processed/kobis_with_credits_ko_genres_exploded.csv'

try:
    df = pd.read_csv(file_path)

    # 1. 장르 데이터 정제 및 분리
    # 먼저 NaN 값을 처리하여 에러 방지
    df['genres'] = df['genres'].fillna('[]')
    df['genres_split'] = df['genres'].str.strip('[]').str.replace("'", "").str.split(', ')

    # 2. 각 장르를 별도의 행으로 분리
    exploded_df = df.explode('genres_split')

    # '장르' 값이 비어있는 행(None) 제거
    exploded_df.dropna(subset=['genres_split'], inplace=True)

    # 3. 새로운 CSV 파일로 저장
    exploded_df.to_csv(output_file_path, index=False)

    print(f"✅ 장르별로 분리된 데이터가 '{output_file_path}'에 성공적으로 저장되었습니다.")
    print("\n저장된 파일의 상위 5개 행:")
    print(exploded_df.head())

except FileNotFoundError:
    print(f"\n오류: '{file_path}' 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")
except KeyError as e:
    print(f"\n오류: {e} 컬럼이 DataFrame에 없습니다. 컬럼명을 다시 확인해주세요.")

✅ 장르별로 분리된 데이터가 'data_processed/kobis_with_credits_ko_genres_exploded.csv'에 성공적으로 저장되었습니다.

저장된 파일의 상위 5개 행:
   movie_id    title          original_title original_language release_date  \
0  20040786  공공의 적 2    Another Public Enemy                ko   2005-01-27   
1  20040785      말아톤                Marathon                ko   2005-01-27   
2  20040774     몽정기2            Wet Dreams 2                ko   2005-01-14   
3  20040775  키다리 아저씨         Daddy-Long-Legs                ko   2005-01-14   
4  20050008  B형 남자친구  My Boyfriend Is Type-B                ko   2005-02-03   

   runtime  budget       revenue  vote_average  vote_count  popularity  \
0    149.0     NaN  2.002252e+10           NaN         NaN         NaN   
1    117.0     NaN  2.603461e+10           NaN         NaN         NaN   
2    101.0     NaN  5.044425e+09           NaN         NaN         NaN   
3     98.0     NaN  2.766454e+09           NaN         NaN         NaN   
4     96.0     NaN  7.005258e+09           NaN

In [13]:
import pandas as pd
from IPython.display import display

# 파일 경로
file_path = 'data_processed/kobis_with_credits_ko_genres_exploded.csv'

try:
    df = pd.read_csv(file_path)

    # 1. 2005년 이후 데이터만 필터링
    df['release_year'] = pd.to_datetime(df['release_date']).dt.year
    recent_df = df[df['release_year'] >= 2005].copy()
    
    # 2. release_date에서 월(month) 추출
    recent_df['release_month'] = pd.to_datetime(recent_df['release_date']).dt.month

    # 3. 장르별 수익 상위 25% 영화 추출
    def get_successful_movies(group):
        q3_revenue = group['revenue'].quantile(0.75)
        return group[group['revenue'] >= q3_revenue]

    successful_movies_df = recent_df.groupby('genres_split').apply(get_successful_movies).reset_index(drop=True)

    # 4. 각 장르별로 그룹화하여 display로 출력
    grouped_by_genre = successful_movies_df.groupby('genres_split')

    print("--- 장르별 흥행 성공 영화 목록 (상위 5개) ---")

    for genre, group_df in grouped_by_genre:
        print(f"\n✅ 장르: {genre}")
        display(group_df[['title', 'revenue', 'audience_total', 'runtime', 'cast_names_ko', 'release_month']].head())
    
except FileNotFoundError:
    print(f"\n오류: '{file_path}' 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")
except KeyError as e:
    print(f"\n오류: {e} 컬럼이 DataFrame에 없습니다. 컬럼명을 다시 확인해주세요.")

--- 장르별 흥행 성공 영화 목록 (상위 5개) ---

✅ 장르: SF


C:\Users\sec\AppData\Local\Temp\ipykernel_20852\3721647741.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  successful_movies_df = recent_df.groupby('genres_split').apply(get_successful_movies).reset_index(drop=True)


,title,revenue,audience_total,runtime,cast_names_ko,release_month
0,괴물,6.656903e+10,10892305.0,119.0,송강호|변희봉|박해일|배두나|고아성,7
1,연가시,3.206548e+10,4499856.0,109.0,김명민|문정희|김동완|이하늬|강신일,7
2,설국열차,6.681845e+10,9321959.0,125.0,크리스 에반스|송강호|에드 해리스|존 허트|틸다 스윈튼,8



✅ 장르: 가족


,title,revenue,audience_total,runtime,cast_names_ko,release_month
3,맨발의 기봉이,1.214021e+10,1988117.0,100.0,신현준|임하룡|탁재훈|김수미|김효진,4
4,괴물,6.656903e+10,10892305.0,119.0,송강호|변희봉|박해일|배두나|고아성,7
5,마이 파더,5.330582e+09,834630.0,105.0,김영철|다니엘 헤니|김인권|이건문|안석환,9
6,파파,3.864042e+09,566746.0,116.0,박용우|고아라|손병호|정명숙|최경선,2
7,깡철이,8.501042e+09,1197467.0,108.0,유아인|김해숙|정유미|김정태|김성오,10


...

,title,revenue,audience_total,runtime,cast_names_ko,release_month
425,웰컴 투 동막골,4.018094e+10,6413223.0,133.0,신하균|정재영|강혜정|리민|류덕환,8
426,신기전,2.422619e+10,3710813.0,134.0,정재영|한은정|허준호|안성기|김명수,9
427,포화속으로,2.369803e+10,3307797.0,120.0,차승원|권상우|최승현|김승우|문재원,6
428,인천상륙작전,5.499344e+10,7033357.0,110.0,이정재|이범수|리암 니슨|진세연|정준호,7



✅ 장르: 코미디


,title,revenue,audience_total,runtime,cast_names_ko,release_month
429,마파도,1.663338e+10,2618070.0,107.0,김수미|여운계|이정진|이문식|오달수,3
430,박수칠 때 떠나라,1.311863e+10,2063656.0,115.0,차승원|신하균|김지수|박정아|조정진,8
431,가문의 위기,2.848571e+10,4508292.0,115.0,신현준|김수미|김원희|정준하|최은주,9
432,광식이 동생 광태,1.276858e+10,2018197.0,104.0,김주혁|봉태규|이요원|김아중|정경호,11
433,투사부일체,3.147477e+10,4997745.0,125.0,정준호|정웅인|김상중|박용식|최윤영,1



✅ 장르: 판타지


,title,revenue,audience_total,runtime,cast_names_ko,release_month
506,디워,4.933355e+10,7854274.0,90.0,브룩스 아만다|베어 제이슨,8
507,전우치,4.378833e+10,6022722.0,136.0,강동원|임수정|김윤석|유해진|김시권,12
508,늑대소년,4.640984e+10,6627275.0,124.0,송중기|박보영|장영남|유연석|김향기,10
509,뷰티 인사이드,1.604508e+10,2039642.0,126.0,한효주|김대명|도지한|배성우|박신혜,8
510,신과함께-죄와 벌,1.156022e+11,14398110.0,139.0,하정우|차태현|주지훈|김향기|김동욱,12
